# [STARTER] Exercise - Create an Agent with external API call enabled

In this exercise, you'll build an agent that can interact with external APIs to gather real-time data
and provide responses based on that information. You'll combine concepts from state management and
memory while adding the ability to make external API calls safely and effectively.


## Challenge

Your task is to create an agent that can make External API Calls:

- Implement tools that interact with real APIs
- Handle API responses and errors gracefully
- Use environment variables for API keys
- Process and format API data for user consumption

## Setup
First, let's import the necessary libraries:

In [1]:
import os
import random
from typing import List
import requests
from dotenv import load_dotenv

from lib.agents import Agent
from lib.messages import BaseMessage
from lib.tooling import tool

In [2]:
load_dotenv()

True

## Define API tools

Feel free to use any open service available through APIs.

Here are a few examples. You can follow the instructions given.
- https://jsonplaceholder.typicode.com/guide/
- https://www.exchangerate-api.com/
- https://openweathermap.org/

Or you can find one you're interested in here.
- https://github.com/public-apis/public-apis

In [11]:
# DONE: Define as many tools that access external APIs as you want
# Example:
@tool
def get_random_pokemon() -> dict:
    """Get a random Pokemon from the original 151"""
    URL = "https://pokeapi.co/api/v2/pokemon?limit=151"
    response = requests.get(URL)
    response.raise_for_status()
    return random.choice(response.json()['results'])

@tool
def get_currency_conversion_rate(base_currency: str, target_currency: str) -> float:
    """
    Retrieve the conversion rate from one currency to another.

    Parameters
    ----------
    base_currency : str
        The three-letter ISO 4217 currency code you want to convert *from*
        (e.g., "USD", "EUR", "JPY").

    target_currency : str
        The three-letter ISO 4217 currency code you want to convert *to*
        (e.g., "GBP", "CAD", "AUD").

    Returns
    -------
    float
        The numerical conversion rate representing how much one unit of
        `base_currency` is worth in `target_currency`.

    Notes
    -----
    - This function should not perform any rounding; return the raw rate.
    - Raise an appropriate exception if currencies are invalid or the
      conversion cannot be retrieved.
    """
    API_KEY = os.getenv("EXCHANGERATE_API_KEY")
    response = requests.get(url=f"https://v6.exchangerate-api.com/v6/{API_KEY}/latest/{base_currency}")

    return response.json()["conversion_rates"][target_currency]

In [18]:
# DONE: Add all the tools you have defined
tools = [get_random_pokemon, get_currency_conversion_rate]

In [19]:
# DONE: Add instructions to your agent

agent = Agent(
    model_name="gpt-4o-mini",
    instructions=(
        "You are an assistant that can help with:\n"
        "choosing random Pokemon\n"
        "calculating exchange rates"
    ),
    tools=tools
)

In [6]:
def print_messages(messages: List[BaseMessage]):
    for m in messages:
        print(f" -> (role = {m.role}, content = {m.content}, tool_calls = {getattr(m, 'tool_calls', None)})")

## Run your Agent

In [16]:
# DONE: Change the query and then run your agent
query = "Give me the conversion rate between USD and BRL."
session_id = "external_tools"

In [20]:
run1 = agent.invoke(
    query=query, 
    session_id=session_id,
)

print("\nMessages from run 1:")
messages = run1.get_final_state()["messages"]
print_messages(messages)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__

Messages from run 1:
 -> (role = system, content = You are an assistant that can help with:
choosing random Pokemon
calculating exchange rates, tool_calls = None)
 -> (role = user, content = Give me the conversion rate between USD and BRL., tool_calls = None)
 -> (role = assistant, content = None, tool_calls = [ChatCompletionMessageToolCall(id='call_jIj45WciYqUbvmtcSsqLbhDu', function=Function(arguments='{"base_currency":"USD","target_currency":"BRL"}', name='get_currency_conversion_rate'), type='function')])
 -> (role = tool, content = "5.3948", tool_calls = None)
 -> (role = assistant, content = The conversion rate from USD to BRL is 5.3948., tool_calls = None)


## Check session histories

In [21]:
runs = agent.get_session_runs(session_id)
for i, run_object in enumerate(runs, 1):
    print(f"\n# Run {i}", run_object.metadata)
    print("Messages:")
    print_messages(run_object.get_final_state()["messages"])


# Run 1 {'run_id': 'd3fa92e7-b071-497d-b2f5-06234da9830c', 'start_timestamp': '2025-11-24 19:00:31.257431', 'end_timestamp': '2025-11-24 19:00:34.130526', 'snapshot_counts': 5}
Messages:
 -> (role = system, content = You are an assistant that can help with:
choosing random Pokemon
calculating exchange rates, tool_calls = None)
 -> (role = user, content = Give me the conversion rate between USD and BRL., tool_calls = None)
 -> (role = assistant, content = None, tool_calls = [ChatCompletionMessageToolCall(id='call_jIj45WciYqUbvmtcSsqLbhDu', function=Function(arguments='{"base_currency":"USD","target_currency":"BRL"}', name='get_currency_conversion_rate'), type='function')])
 -> (role = tool, content = "5.3948", tool_calls = None)
 -> (role = assistant, content = The conversion rate from USD to BRL is 5.3948., tool_calls = None)


In [22]:
runs = agent.get_session_runs(session_id)
for run_object in runs:
    print(run_object)
    for snp in run_object.snapshots:
        print(f"-> {snp}")
    print("\n")

Run('d3fa92e7-b071-497d-b2f5-06234da9830c')
-> Snapshot('fb2e4452-81fc-4df2-9b1c-b7c747b06b81') @ [2025-11-24 19:00:31.257574]: __entry__.State({'user_query': 'Give me the conversion rate between USD and BRL.', 'instructions': 'You are an assistant that can help with:\nchoosing random Pokemon\ncalculating exchange rates', 'messages': [], 'current_tool_calls': None, 'session_id': 'external_tools'})
-> Snapshot('0d07ad17-3812-4e32-9ef3-37ec859f4095') @ [2025-11-24 19:00:31.257713]: message_prep.State({'user_query': 'Give me the conversion rate between USD and BRL.', 'instructions': 'You are an assistant that can help with:\nchoosing random Pokemon\ncalculating exchange rates', 'messages': [SystemMessage(role='system', content='You are an assistant that can help with:\nchoosing random Pokemon\ncalculating exchange rates'), UserMessage(role='user', content='Give me the conversion rate between USD and BRL.')], 'current_tool_calls': None, 'session_id': 'external_tools'})
-> Snapshot('398f87e